In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

In [2]:
import ray
import pandas as pd
from utils import helper, config, rayer
from ml.models.ensemble import Ensemble
from ml.xai.model.explainable import Explainable


from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from ml.models import common


In [3]:
rayer.get_global_cluster(num_cpus=5)
!ray status --address='raycluster-autoscaler-head-svc.dev.svc.cluster.local:6379'

Log channel is reconnecting. Logs produced while the connection was down can be found on the head node of the cluster in `ray_client_server_[port].out`
2022-11-28 04:48:34,321	WARNING dataclient.py:395 -- Encountered connection issues in the data channel. Attempting to reconnect.
2022-11-28 04:48:39,909	ERROR dataclient.py:322 -- Unrecoverable error in data channel.


KeyboardInterrupt: 

In [ ]:
!ray status --address='raycluster-autoscaler-head-svc.dev.svc.cluster.local:6379'

In [6]:
# ray.autoscaler.sdk.request_resources(num_cpus=0)

In [7]:
df_X, df_y = helper.get_covid_dataset()
df_X = df_X.drop(['location'], axis = 1)
# X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.33)

In [8]:
list_base_models = ['BriskXGBoost', 'SlugANN']

ensemble_set = Ensemble(df_X=df_X,
                        df_y=df_y,
                        #list_base_models = list_base_models,
                        n_trials=100, ### for all models
                        boosted_round=100, ### for tree models only
                        epochs=150, ### for DNN models

                        ensemble_boosted_round=10, ### for ensemble model which is also a free model
                        ensemble_n_trials=3,
                                )

In [9]:
threshold=None
ensemble_set.fetch_models(threshold=threshold)
ensemble_set.fit()

2022-11-16T07:17:04CST : INFO : ensemble : fetch_models : 113 : Message : Ensemble: starting discovery process for models [<ml.models.base.brisk_xgboost.BriskXGBoost object at 0x7f69f3e00d90>, <ml.models.base.slug_xgboost.SlugXGBoost object at 0x7f69f3e00640>, <ml.models.base.slug_ann.SlugANN object at 0x7f69f3e00af0>, <ml.models.base.slug_rf.SlugRF object at 0x7f69f3e00b80>, <ml.models.base.slug_knn.SlugKNN object at 0x7f69f3e00b20>, <ml.models.base.brisk_bagging.BriskBagging object at 0x7f69f3e00d60>]
(__run_discoveries__ pid=231, ip=10.10.114.126) 2022-11-16T05:17:07PST : INFO : brisk_xgboost : __discover_model__ : 108 : Message : brisk_xgb: Starting training for trials:100, boosted rounds: 100, max depth: 10
(__run_discoveries__ pid=15199) 2022-11-16T05:17:09PST : INFO : slug_xgboost : __discover_model__ : 154 : Message : slug_xgb: Starting training for trials:100, boosted rounds: 100, max depth: 10
(__run_discoveries__ pid=15199) /tmp/ray/session_2022-11-14_19-18-43_449773_8/runti

(scheduler +16s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +16s) Adding 1 node(s) of type small-group.


(__run_discoveries__ pid=259, ip=10.10.114.126) 2022-11-16T05:17:12PST : INFO : slug_knn : __discover_model__ : 111 : Message : slug_knn: Number of trials: 100
(__run_discoveries__ pid=259, ip=10.10.114.126) 2022-11-16T05:17:12PST : INFO : slug_knn : __discover_model__ : 113 : Message : Best trial:51
(__run_discoveries__ pid=259, ip=10.10.114.126) 2022-11-16T05:17:12PST : INFO : slug_knn : __discover_model__ : 115 : Message :   Params: 
(__run_discoveries__ pid=259, ip=10.10.114.126) 2022-11-16T05:17:12PST : INFO : slug_knn : __discover_model__ : 117 : Message :     n_neighbors 49
(__run_discoveries__ pid=259, ip=10.10.114.126) 2022-11-16T05:17:12PST : INFO : slug_knn : __discover_model__ : 117 : Message :     weights uniform
(__run_discoveries__ pid=259, ip=10.10.114.126) 2022-11-16T05:17:12PST : INFO : slug_knn : __discover_model__ : 117 : Message :     algorithm ball_tree
(__run_discoveries__ pid=259, ip=10.10.114.126) 2022-11-16T05:17:12PST : INFO : slug_knn : __discover_model__ : 

(scheduler +22s) Resized to 7 CPUs.


(__run_discoveries__ pid=259, ip=10.10.114.126) 2022-11-16T05:17:19PST : INFO : brisk_bagging : __discover_model__ : 112 : Message : brisk_bagging: Number of trials: 100
(__run_discoveries__ pid=259, ip=10.10.114.126) 2022-11-16T05:17:19PST : INFO : brisk_bagging : __discover_model__ : 114 : Message : Best trial:13
(__run_discoveries__ pid=259, ip=10.10.114.126) 2022-11-16T05:17:19PST : INFO : brisk_bagging : __discover_model__ : 116 : Message :   Params: 
(__run_discoveries__ pid=259, ip=10.10.114.126) 2022-11-16T05:17:19PST : INFO : brisk_bagging : __discover_model__ : 118 : Message :     n_estimatorsint 14
(__run_discoveries__ pid=259, ip=10.10.114.126) 2022-11-16T05:17:19PST : INFO : brisk_bagging : __discover_model__ : 132 : Message :   test r2 score: 0.6160839566144669
(__run_discoveries__ pid=259, ip=10.10.114.126) 2022-11-16T05:17:19PST : INFO : brisk_bagging : __discover_model__ : 137 : Message : brisk_bagging: Model saved at /tmp/covid_autolearn/brisk_bagging/saved/brisk_bagg

(scheduler +1m27s) Removing 1 nodes of type small-group (idle).
(scheduler +1m39s) Resized to 5 CPUs.


Log channel is reconnecting. Logs produced while the connection was down can be found on the head node of the cluster in `ray_client_server_[port].out`
2022-11-16 07:21:58,938	WARNING dataclient.py:396 -- Encountered connection issues in the data channel. Attempting to reconnect.
2022-11-16 07:22:05,287	ERROR dataclient.py:323 -- Unrecoverable error in data channel.


ConnectionError: GRPC connection failed: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.FAILED_PRECONDITION
	details = "'NoneType' object is not iterable"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:172.20.14.107:10001 {grpc_message:"\'NoneType\' object is not iterable", grpc_status:9, created_time:"2022-11-16T07:22:05.53789929-06:00"}"
>

In [30]:
# ensemble_set.base_models[1].score

In [31]:
# ensemble_set.load_scores()

In [ ]:
ensemble_set.base_model_scores

Unexpected exception:
Traceback (most recent call last):
  File "/home/conda/admin/807c952d1ebb2e498414dc4b80a90b96c80ef2ef433f07c7a7381b5e38e2b5fa-20221110-224801-275852-64-wasif-dev-ray/lib/python3.9/site-packages/ray/util/client/logsclient.py", line 67, in _log_main
    for record in log_stream:
  File "/home/conda/admin/807c952d1ebb2e498414dc4b80a90b96c80ef2ef433f07c7a7381b5e38e2b5fa-20221110-224801-275852-64-wasif-dev-ray/lib/python3.9/site-packages/grpc/_channel.py", line 426, in __next__
    return self._next()
  File "/home/conda/admin/807c952d1ebb2e498414dc4b80a90b96c80ef2ef433f07c7a7381b5e38e2b5fa-20221110-224801-275852-64-wasif-dev-ray/lib/python3.9/site-packages/grpc/_channel.py", line 826, in _next
    raise self
grpc._channel._MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.NOT_FOUND
	details = "Logstream proxy failed to connect. Channel for client a574f55c932844999c2fa4e6b927f2e0 not found."
	debug_error_string = "UNKN

In [ ]:
ensemble_set.score

In [14]:
trained_models = []

for base_model in ensemble_set.base_models:
    if base_model.best_fit:
        trained_models.append(base_model.best_fit)


In [36]:
df_X_id = ray.put(df_X)

In [37]:
df_X_temp = ray.get(df_X_id)

In [40]:
import shap
explainer = shap.Explainer(trained_models[0])
shap_values = explainer(df_X_temp)


TypeError: 'NoneType' object is not callable

(scheduler +32m52s) Resized to 19 CPUs.
(scheduler +32m52s) Removing 2 nodes of type small-group (idle).
(scheduler +33m6s) Resized to 15 CPUs.
(scheduler +33m54s) Removing 5 nodes of type small-group (idle).
(scheduler +34m5s) Resized to 5 CPUs.


In [26]:
models

Sequential(
  (0): Linear(in_features=40, out_features=64, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2996592516681684, inplace=False)
  (3): Linear(in_features=64, out_features=83, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.2996592516681684, inplace=False)
  (6): Linear(in_features=83, out_features=38, bias=True)
  (7): ReLU()
  (8): Dropout(p=0.2996592516681684, inplace=False)
  (9): Linear(in_features=38, out_features=129, bias=True)
  (10): ReLU()
  (11): Dropout(p=0.2996592516681684, inplace=False)
  (12): Linear(in_features=129, out_features=48, bias=True)
  (13): ReLU()
  (14): Dropout(p=0.2996592516681684, inplace=False)
  (15): Linear(in_features=48, out_features=34, bias=True)
  (16): ReLU()
  (17): Dropout(p=0.2996592516681684, inplace=False)
  (18): Linear(in_features=34, out_features=1, bias=True)
  (19): ReLU()
)

In [27]:
models = trained_models[0]

In [ ]:
ex = Explainable(df_X, models)

In [ ]:
attr_algos = ['IG']
ex.get_attr(attr_algos)